**Exercise 2**
<br/>
Design and implement a simple digital certificate framework. Your framework should
allow to create a certificate chain and validate it. (Suppose there are 3 CAs to form
a certificate chain.)


In [8]:
!pip install rsa
!pip install cryptography

In [ ]:
from datetime import datetime, timedelta
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric.padding import PKCS1v15


# Class Entity represent to an Entity with private key and public key
class Entity:
    # init function, generate (private key, public key)
    def __init__(self, public_exponent=65537, key_size=2048):
        self.__keys = rsa.generate_private_key(public_exponent, key_size)

    # as public key is public, this function will return public key if needed
    def get_public_key(self):
        return self.__keys.public_key()


# represent for an Certificate Authenticator


class CertificateAuthority(Entity):

    # init function, save the name, generate private, public keys


    def __init__(self, name: str):
        self.__name = name
        # init keys for Certificate Authority entity (public & private keys)
        super().__init__()

    # generate signature, sign by itself
    def generate_signature(self, subject_name, public_key):
        # requestor name
        subject = x509.Name(
            [
                x509.NameAttribute(NameOID.ORGANIZATION_NAME, subject_name),
                x509.NameAttribute(NameOID.COMMON_NAME, subject_name),
            ]
        )

        # CA name
        issuer = x509.Name(
            [
                x509.NameAttribute(NameOID.ORGANIZATION_NAME, self.__name),
                x509.NameAttribute(NameOID.COMMON_NAME, self.__name),
            ]
        )

        # form a certificate object to the builder
        certificate = (
            x509.CertificateBuilder()
            .subject_name(subject)
            .issuer_name(issuer)
            .public_key(public_key)
            .serial_number(x509.random_serial_number())
            .not_valid_before(datetime.utcnow())
            .not_valid_after(datetime.utcnow() + timedelta(days=365))
            .sign(self._Entity__keys, hashes.SHA256())
        )
        return certificate

    # verify certificate based on its public key
    def verify_certificate(self, certificate):
        try:
            self.get_public_key().verify(
                certificate.signature,
                certificate.tbs_certificate_bytes,
                PKCS1v15(),
                certificate.signature_hash_algorithm,
            )
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False


def get_certificate(cert):
    return cert.public_bytes(serialization.Encoding.PEM).decode()


# initial root CA & intermediate CA


root_ca = CertificateAuthority("Root CA")
# root cert generate by itself
root_ca_cert = root_ca.generate_signature("Root CA", root_ca.get_public_key())
print(f"Root CA certificate: \n {get_certificate(root_ca_cert)}")

intermediate_ca = CertificateAuthority("Intermediate CA")
# intermediate cert generated by root CA
intermediate_ca_cert = root_ca.generate_signature(
    "Intermediate CA", intermediate_ca.get_public_key()
)
print(f"\nIntermediate CA certificate: \n {get_certificate(intermediate_ca_cert)}")

alice = Entity()
# Alice cert generated by intermediate CA
cert = intermediate_ca.generate_signature("Alice", alice.get_public_key())
print(f"\nAlice certificate: \n {get_certificate(cert)}")
# validate Alice's certificate
if intermediate_ca.verify_certificate(cert):
    print("Verified Alice certificate")
else:
    print("Alice certificate is unauthorized")

Root CA certificate: 
 -----BEGIN CERTIFICATE-----
MIIC1DCCAbygAwIBAgIUGZEdR8pWYSU7i9C0ofqpw76+KuEwDQYJKoZIhvcNAQEL
BQAwJDEQMA4GA1UECgwHUm9vdCBDQTEQMA4GA1UEAwwHUm9vdCBDQTAeFw0yNDEy
MTMxODIxMzFaFw0yNTEyMTMxODIxMzFaMCQxEDAOBgNVBAoMB1Jvb3QgQ0ExEDAO
BgNVBAMMB1Jvb3QgQ0EwggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQCj
wkOXppSV0kzUc9Xf057qAYwMtT3rNNTfi2cFbnqRftnxFHcJVLDh7yFAJhPFU4Ei
nS8qdF4mXpxWq95IanQlIqertDKm1hj4DT7V2BWNYrQEqHWUbyA3ahLwnAOaUUtX
7xSEcKDpdDTa8uFF2VbKAxJmTxHa21SMmseYA95qklJ5NkKEwvV94HqsrcBQDokc
hp4/1jUm21jIeosVv/v0edJjaMx47xrLIlmQGUPNOpKhByNvd7TICY4icyFarUJg
93Soh7DvaKf7z9oVzXQdmIYj0i7nq5+srqV69FMD1WTNqm/YRhe9UgQIqQwsc7kM
hHnmGmMHgEpraEUoV9yFAgMBAAEwDQYJKoZIhvcNAQELBQADggEBAJIy8wdnUaWn
MMVnC6WUFNyCOYV6+JAFRwe6P1+rzu2iF55f6bVeM0H1+MqDHri8MBnKOeP0m/AC
g0h8KhRWI6j71P33+KvPqOHU7CuVctr4hb1MuY8HYKMwPxFZFKBPyodbhm9mGIxx
CQq4g39AXnE4II4O8WLyGmgt0Ab/S4kOvQaz17O68qwjKpoOlwWhNwd7GKSeOnEw
acdzyI/dxs2WZLF1QcmuiU54h1G+aLszDamCj+tUG4rv4zAvLKIBUvM7VF60dStO
9tIteZSzr+WJE7qOCww990FVtyHt3Y42kOqSs9Z